## Código para identificar tiendas Cornershop por código postal

Con base en los códigos postales de SEPOMEX, se identifican las tiendas Cornershop que se encuentran en cada uno de los códigos postales.

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import time
import requests
import random

In [6]:
#Directorio de trabajo
try:
    os.chdir('C:/Users/claudio.pacheco/Documents/GitHub/pacic/')
except:
    os.chdir("D:/")

#### Descarga de datos de Sepomex

In [7]:
# #Descarga de códigos postales de Ciudad de México
# url = 'https://www.correosdemexico.gob.mx/SSLServicios/ConsultaCP/CodigoPostal_Exportar.aspx'
# driver = webdriver.Chrome("C:/chromedriver.exe")
# driver.get(url)
# combo = driver.find_element(By.CLASS_NAME, "cbo")
# select = Select(combo)
# select.select_by_visible_text("Jalisco")
# #Descarga en carpeta de trabajo
# driver.find_element(By.ID, "btnDescarga").click()
# time.sleep(5)
# driver.close()


In [6]:

#Mover a carpeta de trabajo el archivo descargado

#os.rename('C:/Users/claudio.pacheco/Downloads/Ciudad de México.xls', 'C:/Users/claudio.pacheco/Documents/GitHub/pacic/cp/Ciudad de México.xls')

In [11]:
#Leer archivo descargado
cp = pd.read_excel('cp/Jalisco.xls',sheet_name="Jalisco",usecols=["d_codigo", "d_asenta", 
                                                                                    "d_tipo_asenta", "D_mnpio", 
                                                                                    "c_mnpio","d_zona"],dtype={"d_codigo":str})
cp.columns=cp.columns.str.lower()

In [12]:
cp

,d_codigo,d_asenta,d_tipo_asenta,d_mnpio,c_mnpio,d_zona
0,44100,Guadalajara Centro,Colonia,Guadalajara,39,Urbano
1,44110,Vallarta Poniente,Fraccionamiento,Guadalajara,39,Urbano
2,44130,Arcos Vallarta,Colonia,Guadalajara,39,Urbano
3,44130,Arcos,Fraccionamiento,Guadalajara,39,Urbano
4,44140,Obrera Centro,Colonia,Guadalajara,39,Urbano
...,...,...,...,...,...,...
6044,49990,Villa Morelos (El Carrizo),Ranchería,Santa María del Oro,56,Rural
6045,49990,Los Plátanos,Colonia,Santa María del Oro,56,Rural
6046,49994,Zipoco,Ranchería,Santa María del Oro,56,Rural
6047,49994,Las Parotas,Ranchería,Santa María del Oro,56,Rural


In [13]:
#Eliminar duplicados
cp = cp.drop_duplicates(subset=['d_codigo'], keep='first')

In [14]:
#Dejar un código postal por c_mnpio
cp = cp.drop_duplicates(subset=['c_mnpio'], keep='first')
cp

,d_codigo,d_asenta,d_tipo_asenta,d_mnpio,c_mnpio,d_zona
0,44100,Guadalajara Centro,Colonia,Guadalajara,39,Urbano
438,45010,Paseos del Parque,Fraccionamiento,Zapopan,120,Urbano
1115,45250,San Cristóbal de La Barranca,Pueblo,San Cristóbal de la Barranca,71,Rural
1123,45260,Ixtlahuacán del Río,Pueblo,Ixtlahuacán del Río,45,Urbano
1141,45300,El Venado,Colonia,Tala,83,Urbano
...,...,...,...,...,...,...
5906,49840,Tonila,Pueblo,Tonila,103,Rural
5911,49870,Pihuamo,Colonia,Pihuamo,65,Urbano
6003,49900,La Campana,Colonia,Tecalitlán,87,Urbano
6023,49950,Lomas Altas,Colonia,Jilotlán de los Dolores,49,Urbano


In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


#Parámetros
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

options=webdriver.EdgeOptions()
options.add_argument("-inprivate")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-cache")
options.add_argument("--disable-cookies")

In [26]:
def data_walmart(zip_code,busqueda):
    busqueda=busqueda.replace(" ","+")

    #Cambia ubicación para obtener precios
    url="https://super.walmart.com.mx/"
    url_busqueda_template="https://super.walmart.com.mx/search?q="+busqueda+"&page={}"
    driver=webdriver.Edge(options=options)
    driver.get(url)
    time.sleep(5)
    try:
        elemento=driver.find_element(By.NAME,"select-store-button")
    except NoSuchElementException:
        elemento = driver.find_element(By.CSS_SELECTOR, 'img[src="https://i5.walmartimages.com/dfw/4ff9c6c9-14f0/k2-_8ce34043-6050-4f39-afb2-3e76a554f8a3.v1.png"]')
    elemento.click()
    #Ingresa código postal
    time.sleep(4)
    cp=driver.find_element(By.CSS_SELECTOR,"input[data-automation-id='store-zip-code']")
    cp.clear()
    cp.send_keys(zip_code)
    time.sleep(4)
    #Obtener datos de la tienda
    tienda = driver.find_element(By.CSS_SELECTOR, 'span.b.f5.lh-copy.dark-gray').text
    address = driver.find_element(By.CSS_SELECTOR, 'span.f6[aria-label]').get_attribute('aria-label')
    radio_buttons=driver.find_element(By.NAME,'pickup-store')
    #Aceptar elección
    elegir=driver.find_element(By.CSS_SELECTOR,"button[data-automation-id='save-label']")
    elegir.click()
    time.sleep(4)

    all_data=[]
    for page_number in range(1,2):
        url_busqueda=url_busqueda_template.format(page_number)
        driver.get(url_busqueda)
        time.sleep(4)
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        results = soup.find_all('div', {'data-stack-index': '0'})
        span_tags = []
        product_titles = []
        # Loop para obtener precios y productos
        for result in results:
            span_tags.extend(result.select('div[data-automation-id="product-price"] span'))
            product_titles.extend([title.text for title in result.select('span[data-automation-id="product-title"]')])

        precios = []

        for span in span_tags:
            text = span.text
            if 'precio actual' in text:
                precios.append(text.split("precio actual")[1].strip())
        #Fecha y hora de la consulta
        fecha=time.strftime("%d/%m/%y")
        hora=time.strftime("%H:%M:%S")
        fecha_hora=hora+" "+fecha
        dfx=pd.DataFrame({'producto':product_titles,'precio':precios,'tienda':tienda,'direccion':address,'busqueda':busqueda,'fecha_consulta':fecha_hora})
        all_data.append(dfx)
        time.sleep(5)
    df=pd.concat(all_data,ignore_index=True)
    #Limpiar precios
    #Dejar solo números utilizando expresiones regulares
    df['precio'] = df['precio'].str.replace(r'[^0-9.]', '', regex=True)
    #Eliminar comas
    df['precio']=df['precio'].str.replace(',','',regex=True)
    #Convertir a numérico
    df['precio']=pd.to_numeric(df['precio'])
    df['producto']=df.producto.str.lower()
    return df
    

In [63]:
muestra=cp.sample(12)
muestra

,d_codigo,d_asenta,d_tipo_asenta,d_mnpio,c_mnpio,d_zona
5688,49500,La Cofradía,Ejido,Mazamitla,59,Rural
5659,49370,Amacueca,Pueblo,Amacueca,4,Rural
0,44100,Guadalajara Centro,Colonia,Guadalajara,39,Urbano
2683,46200,Colotlan Centro,Colonia,Colotlán,25,Semiurbano
4095,47540,Magisterial,Colonia,Ojuelos de Jalisco,64,Urbano
4650,48150,Cuautla,Pueblo,Cuautla,28,Rural
5562,49120,Residencial El Paraje,Fraccionamiento,Gómez Farías,79,Rural
5664,49400,Francisco Dávalos Flores,Colonia,Tizapán el Alto,96,Rural
3151,47140,San Miguel El Alto Centro,Colonia,San Miguel el Alto,78,Semiurbano
5572,49170,Concepción de Buenos Aires,Pueblo,Concepción de Buenos Aires,26,Rural


In [78]:
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup
import pandas as pd

def data_walmart(zip_code, busqueda):
    busqueda = busqueda.replace(" ", "+")
    
    #Cambia ubicación para obtener precios
    url = "https://super.walmart.com.mx/"
    url_busqueda_template = "https://super.walmart.com.mx/search?q=" + busqueda + "&page={}"
    
    driver = webdriver.Edge(options=options)
    driver.get(url)
    time.sleep(5)
    
    try:
        elemento = driver.find_element(By.NAME,"select-store-button")
    except NoSuchElementException:
        elemento = driver.find_element(By.CSS_SELECTOR, 'img[src="https://i5.walmartimages.com/dfw/4ff9c6c9-14f0/k2-_8ce34043-6050-4f39-afb2-3e76a554f8a3.v1.png"]')
    elemento.click()

    #Ingresa código postal
    time.sleep(4)
    cp = driver.find_element(By.CSS_SELECTOR, "input[data-automation-id='store-zip-code']")
    cp.clear()
    cp.send_keys(zip_code)
    time.sleep(4)
    
    # Attempt to retrieve store details
    try:
        tienda = driver.find_element(By.CSS_SELECTOR, 'span.b.f5.lh-copy.dark-gray').text
        address = driver.find_element(By.CSS_SELECTOR, 'span.f6[aria-label]').get_attribute('aria-label')
        radio_buttons = driver.find_element(By.NAME, 'pickup-store')
    except NoSuchElementException:
        # If any of the above elements are not found, return an empty DataFrame.
        print(f"No existe Walmart cercano al código postal {zip_code}.")
        return pd.DataFrame()

    #Aceptar elección
    elegir = driver.find_element(By.CSS_SELECTOR, "button[data-automation-id='save-label']")
    elegir.click()
    time.sleep(4)

    all_data = []
    for page_number in range(1, 2):
        url_busqueda = url_busqueda_template.format(page_number)
        driver.get(url_busqueda)
        time.sleep(4)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        results = soup.find_all('div', {'data-stack-index': '0'})
        
        span_tags = []
        product_titles = []
        
        # Loop para obtener precios y productos
        for result in results:
            span_tags.extend(result.select('div[data-automation-id="product-price"] span'))
            product_titles.extend([title.text for title in result.select('span[data-automation-id="product-title"]')])

        precios = []
        for span in span_tags:
            text = span.text
            if 'precio actual' in text:
                precios.append(text.split("precio actual")[1].strip())

        fecha = time.strftime("%d/%m/%y")
        hora = time.strftime("%H:%M:%S")
        fecha_hora = hora + " " + fecha

        dfx = pd.DataFrame({
            'producto': product_titles,
            'precio': precios,
            'tienda': tienda,
            'direccion': address,
            'busqueda': busqueda,
            'fecha_consulta': fecha_hora
        })
        
        all_data.append(dfx)
        time.sleep(5)

    df = pd.concat(all_data, ignore_index=True)
    
    # Limpiar precios
    df['precio'] = df['precio'].str.replace(r'[^0-9.]', '', regex=True)  # Dejar solo números utilizando expresiones regulares
    df['precio'] = df['precio'].str.replace(',', '', regex=True)  # Eliminar comas
    df['precio'] = pd.to_numeric(df['precio'])
    df['producto'] = df.producto.str.lower()

    return df



In [81]:
data_walmart('44100','azucar')

,producto,precio,tienda,direccion,busqueda,fecha_consulta
0,azúcar morena zulka 1 kg,42.0,16 De Septiembre,"Calz Independencia Sur 916, Mexicaltzingo, Gua...",azucar,13:41:09 01/09/23
1,azúcar aurrera estándar 1 kg,27.0,16 De Septiembre,"Calz Independencia Sur 916, Mexicaltzingo, Gua...",azucar,13:41:09 01/09/23
2,azúcar mascabado metco genuino 1.2 kg,67.5,16 De Septiembre,"Calz Independencia Sur 916, Mexicaltzingo, Gua...",azucar,13:41:09 01/09/23
3,azúcar morena zulka 2 kg,74.5,16 De Septiembre,"Calz Independencia Sur 916, Mexicaltzingo, Gua...",azucar,13:41:09 01/09/23
4,azúcar mascabado great value 1 kg,37.0,16 De Septiembre,"Calz Independencia Sur 916, Mexicaltzingo, Gua...",azucar,13:41:09 01/09/23
5,azúcar mascabado zulka 500 g,25.5,16 De Septiembre,"Calz Independencia Sur 916, Mexicaltzingo, Gua...",azucar,13:41:09 01/09/23
6,azúcar great value refinada 2 kg,71.0,16 De Septiembre,"Calz Independencia Sur 916, Mexicaltzingo, Gua...",azucar,13:41:09 01/09/23
7,azúcar mascabado metco genuino 60 sobres de 4 ...,24.0,16 De Septiembre,"Calz Independencia Sur 916, Mexicaltzingo, Gua...",azucar,13:41:09 01/09/23
8,azúcar glass zulka 500 g,24.5,16 De Septiembre,"Calz Independencia Sur 916, Mexicaltzingo, Gua...",azucar,13:41:09 01/09/23
9,bebida lala vita coco sin azúcar 960 ml,33.5,16 De Septiembre,"Calz Independencia Sur 916, Mexicaltzingo, Gua...",azucar,13:41:09 01/09/23


In [64]:
azucar = pd.DataFrame()

for codigo in muestra['d_codigo']:
    try:
        df = data_walmart(codigo, "azucar")
        azucar = pd.concat([azucar, df], ignore_index=True)
    except Exception as e:
        print(f"Error for codigo {codigo}: {e}")
        continue


Error for codigo 49500: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.b.f5.lh-copy.dark-gray"}
  (Session info: MicrosoftEdge=116.0.1938.62)
Stacktrace:
	GetHandleVerifier [0x00007FF7393A3DB2+61490]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF739336002+740642]
	(No symbol) [0x00007FF73910B8AE]
	(No symbol) [0x00007FF7391532F4]
	(No symbol) [0x00007FF739153405]
	(No symbol) [0x00007FF73918FCE7]
	(No symbol) [0x00007FF73917394F]
	(No symbol) [0x00007FF739147021]
	(No symbol) [0x00007FF73918CE21]
	(No symbol) [0x00007FF7391736E3]
	(No symbol) [0x00007FF739145EAA]
	(No symbol) [0x00007FF73914518B]
	(No symbol) [0x00007FF739146634]
	Microsoft::Applications::Events::ILogManager::DispatchEventBroadcast [0x00007FF739568D69+1207369]
	(No symbol) [0x00007FF7391C5304]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF7392890F1+32273]
	Microsoft::Applications::Events::EventProperty::~EventProperty

In [65]:
azucar

,producto,precio,tienda,direccion,busqueda,fecha_consulta
0,azúcar morena zulka 1 kg,42.0,Chapala Ajijic,"Paseo Centenario De La Revolución No. 20, Colo...",azucar,11:57:18 01/09/23
1,azúcar aurrera estándar 1 kg,27.0,Chapala Ajijic,"Paseo Centenario De La Revolución No. 20, Colo...",azucar,11:57:18 01/09/23
2,azúcar mascabado metco genuino 1.2 kg,67.5,Chapala Ajijic,"Paseo Centenario De La Revolución No. 20, Colo...",azucar,11:57:18 01/09/23
3,azúcar morena zulka 2 kg,74.5,Chapala Ajijic,"Paseo Centenario De La Revolución No. 20, Colo...",azucar,11:57:18 01/09/23
4,azúcar mascabado great value 1 kg,37.0,Chapala Ajijic,"Paseo Centenario De La Revolución No. 20, Colo...",azucar,11:57:18 01/09/23
...,...,...,...,...,...,...
85,endulzante splenda azúcar con stevia 500 g,40.0,Chapala Ajijic,"Paseo Centenario De La Revolución No. 20, Colo...",azucar,11:58:30 01/09/23
86,media crema azucarada chantilly chanty wip azu...,66.0,Chapala Ajijic,"Paseo Centenario De La Revolución No. 20, Colo...",azucar,11:58:30 01/09/23
87,perlas de azúcar moctezuma para decorar multic...,29.0,Chapala Ajijic,"Paseo Centenario De La Revolución No. 20, Colo...",azucar,11:58:30 01/09/23
88,sprinkle de azúcar moctezuma para decorar mult...,29.0,Chapala Ajijic,"Paseo Centenario De La Revolución No. 20, Colo...",azucar,11:58:30 01/09/23


In [72]:
azucar.to_csv('azucar.csv',index=False,encoding='latin1')

In [77]:
copia=azucar.copy()
#Contar número de veces que aparece cada producto
copia['producto'].value_counts().sort_values(ascending=False)


azúcar morena zulka 1 kg                                      2
azúcar aurrera estándar 1 kg                                  2
azúcar mascabado metco genuino 1.2 kg                         2
azúcar morena zulka 2 kg                                      2
azúcar mascabado great value 1 kg                             2
azúcar mascabado zulka 500 g                                  2
azúcar great value refinada 2 kg                              2
azúcar mascabado metco genuino 60 sobres de 4 g c/u           2
azúcar glass zulka 500 g                                      2
alimento líquido silk de soya sin azúcar 946 ml               2
azúcar glass great value 1 kg                                 2
azúcar mascabado metco genuino 690 g                          2
azúcar aires de campo orgánica estándar 1 kg                  2
azúcar de coco a de coco orgánico 200 g                       2
azúcar de coco nbf orgánica 300 g                             2
azúcar domino refinada 50 sobres de 4 g 

In [ ]:
#Contar frecuencia de products